In [3]:
from extract import SocialETL, Count, UserETL
import hashtags as h
import pandas as pd
import warnings 
warnings.filterwarnings('always')   #suppresses all warnings
warnings.filterwarnings('ignore')

## 1 parte: categorizzazione hashtags per prorus, proukr, pax

## 2 parte: scaricare df e distinzione 4 cat (no care) dalle altre

In [38]:
#scarcare df
m = SocialETL(query="(#eurovision OR #covid OR #pridemonth OR #grammys OR #justiceforjohnnydepp OR #oscars OR #elonmusk OR #putin OR #russia OR #ukraine OR #war)  lang:en", pages=1, recent=False)#strangerthings4 OR onepiece

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

In [39]:
m.df

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.geo.bbox,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,
0,1544992379815399424,1544992379815399424,NaN,1544982839434903552,NaN,2312810112,NaN,14106476,NaN,2022-07-07T10:31:02.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07T10:31:14+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
1,1544992377860804608,1544992377860804608,NaN,1544883547709718528,NaN,1386674395511287816,NaN,582941136,NaN,2022-07-07T10:31:02.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07T10:31:14+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
2,1544992373347958784,1544992373347958784,NaN,NaN,NaN,47941897,NaN,NaN,NaN,2022-07-07T10:31:01.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07T10:31:14+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
3,1544992370965377024,1544992370965377024,NaN,NaN,NaN,1544985126068977665,NaN,NaN,NaN,2022-07-07T10:31:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07T10:31:14+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
4,1544992370139078656,1544992370139078656,NaN,NaN,NaN,78576733,NaN,NaN,NaN,2022-07-07T10:31:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07T10:31:14+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1544991970460057601,1544991970460057601,NaN,NaN,1544983822713978882,1456228393234493446,NaN,NaN,1295405245011460096,2022-07-07T10:29:24.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07T10:31:14+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
96,1544991965745336320,1544991965745336320,NaN,1539899508879986688,NaN,1522455425471328256,NaN,1519982389971152896,NaN,2022-07-07T10:29:23.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07T10:31:14+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
97,1544991959399501829,1544991959399501829,NaN,NaN,NaN,19734484,NaN,NaN,NaN,2022-07-07T10:29:22.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07T10:31:14+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
98,1544991957940076544,1544991957940076544,NaN,1544971395528097795,NaN,2988731651,NaN,1495480590572961792,NaN,2022-07-07T10:29:21.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-07T10:31:14+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN


In [40]:
user_id=m.df['author_id']

print(user_id)

0              2312810112
1     1386674395511287816
2                47941897
3     1544985126068977665
4                78576733
             ...         
95    1456228393234493446
96    1522455425471328256
97               19734484
98             2988731651
99               82502558
Name: author_id, Length: 100, dtype: object


In [42]:
#estarre pure retweeted_user_id per edge
hashtag_for_user={}
retweet_for_user={}
verified_for_user={}
location_for_user={}
name_for_user={}
username_for_user={}

for e in user_id:
    u=UserETL(e,pages=1)                                                       #aggiungere più pagine (1pag=20)
    #preprocessing hashtag
    u.df=u.df.dropna(subset=['entities.hashtags'])                             #drop nan values on column hashtags
    u.df.reset_index()
    u.df['entities.hashtags']=u.df['entities.hashtags'].apply(eval)            #convert into dict
    u.df["entities.hashtags"] = u.df["entities.hashtags"].map(h.extract_tags)  #extract tags
    hashtag_for_user[e]=u.df["entities.hashtags"].tolist()                     #convert pandas series into a list
    #preprocessing retweet
    u.df=u.df.dropna(subset=['retweeted_user_id'])
    retweet_for_user[e]=u.df['retweeted_user_id'].tolist()
    #saving other attributes (to assigne it to a nodes as property)
    verified_for_user[e]=u.df['author.verified']
    location_for_user[e]=u.df['author.location'].tolist()
    name_for_user[e]=u.df['author.name']
    name_for_user[e]=u.df['author.username']
    
    
    

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

...

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

In [55]:
user=[]
hashtag=[]
retweet=[]
for u,h in hashtag_for_user.items():
    user.append(u)
    hashtag.append(h)
for u,r in retweet_for_user.items():
    retweet_list.append(r)


In [63]:
#create a pandas dataframe
df_IDK1=pd.DataFrame({"user_id":user,
                      'retweet': retweet,
                        "hashtag":hashtag})
df_IDK1

,user_id,retweet,hashtag
0,2312810112,"[14106476, 1219232377605644289, 14981764520842...","[[sweden, ukraine], [kharkiv, russia, ukraine]..."
1,1386674395511287816,"[1394110787023560707, 1394110787023560707, 582...","[[kashmirdiaries, womenempowerment, indianarmy..."
2,47941897,"[385326089, 923231030160400384, 92323103016040...","[[russia], [assange, afghanistan], [donbass], ..."
3,1544985126068977665,[],"[[covid19, covid, covid, omicron], [covid19, c..."
4,78576733,"[16973333, 267855388]","[[ukraine], [edinburgh, books], [jkrowling, ma..."
...,...,...,...
73,23820751,"[1219232377605644289, 1219232377605644289, 121...","[[russia], [dnipro, russia, putin], [russia, u..."
74,1522455425471328256,"[1519982389971152896, 1519982389971152896, 151...","[[pulwama, kashmir, kashmiragainstterror, fatf..."
75,19734484,[],"[[ukraine], [traitors], [toryscum]]"
76,2988731651,"[1134075064276606976, 1137742952195735553, 324...","[[], [500, , ], [], [ukraine, sumy], [, , 112]..."


In [64]:
#convert nested lists on set
outer_list = [] 
def lists_to_list(nested_lists): 
    for el in nested_lists: 
        if type(el) == list: 
            lists_to_list(el) 
        else: 
            outer_list.append(el)
    return set(outer_list)

In [65]:
outer_list = [] 
df_IDK1['hashtag']=df_IDK1['hashtag'].map(lists_to_list)
df_IDK1

,user_id,retweet,hashtag
0,2312810112,"[14106476, 1219232377605644289, 14981764520842...","{russiancolonialism, kharkiv, ukraine, sweden,..."
1,1386674395511287816,"[1394110787023560707, 1394110787023560707, 582...","{saudi, ukraine, badaltajammukashmir, russia, ..."
2,47941897,"[385326089, 923231030160400384, 92323103016040...","{saudi, slovyansk, russian, ukraine, badaltaja..."
3,1544985126068977665,[],"{saudi, slovyansk, russian, ukraine, badaltaja..."
4,78576733,"[16973333, 267855388]","{saudi, slovyansk, russian, ukraine, badaltaja..."
...,...,...,...
73,23820751,"[1219232377605644289, 1219232377605644289, 121...","{linke, , healthyeating, adayinthelife, maga, ..."
74,1522455425471328256,"[1519982389971152896, 1519982389971152896, 151...","{linke, , healthyeating, adayinthelife, maga, ..."
75,19734484,[],"{linke, , healthyeating, adayinthelife, maga, ..."
76,2988731651,"[1134075064276606976, 1137742952195735553, 324...","{linke, , healthyeating, adayinthelife, maga, ..."


In [66]:
outer_list = [] 
df_IDK1['retweet']=df_IDK1['retweet'].map(lists_to_list)
df_IDK1

,user_id,retweet,hashtag
0,2312810112,"{1498176452084256772, 1219232377605644289, 135...","{russiancolonialism, kharkiv, ukraine, sweden,..."
1,1386674395511287816,"{1359617172033724416, 14106476, 14981764520842...","{saudi, ukraine, badaltajammukashmir, russia, ..."
2,47941897,"{1521787270260719616, 923231030160400384, 1359...","{saudi, slovyansk, russian, ukraine, badaltaja..."
3,1544985126068977665,"{1521787270260719616, 923231030160400384, 1359...","{saudi, slovyansk, russian, ukraine, badaltaja..."
4,78576733,"{1521787270260719616, 923231030160400384, 1359...","{saudi, slovyansk, russian, ukraine, badaltaja..."
...,...,...,...
73,23820751,"{720139699, 14106476, 1458038317619073044, 316...","{linke, , healthyeating, adayinthelife, maga, ..."
74,1522455425471328256,"{720139699, 14106476, 1458038317619073044, 316...","{linke, , healthyeating, adayinthelife, maga, ..."
75,19734484,"{720139699, 14106476, 1458038317619073044, 316...","{linke, , healthyeating, adayinthelife, maga, ..."
76,2988731651,"{720139699, 14106476, 1458038317619073044, 316...","{linke, , healthyeating, adayinthelife, maga, ..."


In [46]:
df_IDK1

,user_id,retweet,hashtag
0,2312810112,"{linke, , healthyeating, adayinthelife, maga, ...","{russiancolonialism, kharkiv, ukraine, sweden,..."
1,1386674395511287816,"{linke, , healthyeating, adayinthelife, maga, ...","{saudi, ukraine, badaltajammukashmir, russia, ..."
2,47941897,"{linke, , healthyeating, adayinthelife, maga, ...","{saudi, slovyansk, russian, ukraine, badaltaja..."
3,1544985126068977665,"{linke, , healthyeating, adayinthelife, maga, ...","{saudi, slovyansk, russian, ukraine, badaltaja..."
4,78576733,"{linke, , healthyeating, adayinthelife, maga, ...","{saudi, slovyansk, russian, ukraine, badaltaja..."
...,...,...,...
73,23820751,"{linke, , healthyeating, adayinthelife, maga, ...","{linke, , healthyeating, adayinthelife, maga, ..."
74,1522455425471328256,"{linke, , healthyeating, adayinthelife, maga, ...","{linke, , healthyeating, adayinthelife, maga, ..."
75,19734484,"{linke, , healthyeating, adayinthelife, maga, ...","{linke, , healthyeating, adayinthelife, maga, ..."
76,2988731651,"{linke, , healthyeating, adayinthelife, maga, ...","{linke, , healthyeating, adayinthelife, maga, ..."


In [37]:
proukr=set(["slavaukraini"]) #1 cat
prorus=set(["istandwithputin", "istandwithrussia"]) #2 cat
pax=set(["stopwarinukraine"]) #3 cat
cat=set.union(proukr,prorus,pax)
count1=0
#count2=0
#count3=0
count4=0
index1=[]
#index2=[]
#index3=[]
index4=[]

for i, row in enumerate(df_IDK1['hashtag']): 
    if row.intersection(cat)==set():#no intersezione   
        #print(row.intersectionn(cat))
        #print("l'utente {}  appartiene alla 4 categoria".format(row.intersection(cat)))
        #print("4 categoria", row)
        count4 += 1
        index4.append(i)
        
    else: #c'è intersezione: uno della 3 categorie
        #print("non è la 4 categoria perchè contiene",row.intersection(cat))
       # if row.intersection(proukr)!=set(): #c'è intersez con proukr
           # count1 += 1
            index1.append(i) #aggiungo alle rige da tgliere
            print("l'utente {} non  appartiene alla 4 categoria perchè usa {}".format(i,row.intersection(cat)))
       # elif row.intersection(prorus)!=set():
          #  count2 += 1
           # index2.append(i)
        #elif row.intersection(pax)!=set():
           # count3 += 1
           #index3.append(i)

l'utente 28 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 29 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 30 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 31 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 32 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 33 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 34 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 35 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 36 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 37 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 38 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 39 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 40 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 41 non  appartiene alla 4 categoria perchè

In [29]:
print('proukr = {} \n'
     'prourus = {} \n'
     'pax = {} \n'
     'nocare = {} '.format(count1,count2,count3,count4))

proukr = 391 
prourus = 0 
pax = 0 
nocare = 28 


## 3 parte categorizzazione tweet e utenti di proukr, prorus, pax

## 4 parte: edge tra utenti nel dataset

In [67]:
df_IDK1['retweet']

0     {1498176452084256772, 1219232377605644289, 135...
1     {1359617172033724416, 14106476, 14981764520842...
2     {1521787270260719616, 923231030160400384, 1359...
3     {1521787270260719616, 923231030160400384, 1359...
4     {1521787270260719616, 923231030160400384, 1359...
                            ...                        
73    {720139699, 14106476, 1458038317619073044, 316...
74    {720139699, 14106476, 1458038317619073044, 316...
75    {720139699, 14106476, 1458038317619073044, 316...
76    {720139699, 14106476, 1458038317619073044, 316...
77    {720139699, 14106476, 1458038317619073044, 316...
Name: retweet, Length: 78, dtype: object